# SL2P-CCRS-S2-downscaling
A python verison of the Simplified Level 2 Prototype Processor (SL2P) for mapping vegetation variables (leaf area index (LAI), fraction of canopy cover (fCOVER), fraction of absorbed photosynthetically active radiation (fAPAR) and surface albedo) at 10 or 20 m spatial resolution from Sentinel-2 MSI L2A images.  

Cite as Djamai N. et al., 2024, "SL2P-PYTHON", Canada Centre for Remote Sensing, DOI: 10.5281/zenodo.10654520,https://github.com/djamainajib/SL2P_PYTHON.

SL2P-PYTHON provides identical results as the CCRS implementation of SL2P within the [LEAF-Toolbox](https://code.earthengine.google.com/8ee611fad1609740099eabbfe571189c) that follows the algorithm theoretical basis document of Weiss and Baret (2016). 

SL2P-PYTHON currently supports SL2PV0 algorithm: defined by Weiss and Baret (2016) with an update in Weiss and Baret (2020).

* [Fernandes R., Brown L., Canisius F., Dash J., He L.M., Hong G., Huang L., Le N.Q., MacDougall C., Meier C., Darko P.O., Shah H., Spafford L., Sun L.X., 2023](https://www.sciencedirect.com/science/article/pii/S0034425723001517). Validation of Simplified Level 2 Prototype Processor Sentinel-2 fraction of canopy cover, fraction of absorbed photosynthetically active radiation and leaf area index products over north American forests. Remote Sens. Environ. 293, 113600. 

* [Weiss M. and Baret F.,2016](https://step.esa.int/docs/extra/ATBD_S2ToolBox_L2B_V1.1.pdf). S2ToolBox Level 2 products: LAI, FAPAR, FCOVER, 1.1. ed. Institut National de la Recherche Agronomique, Avignon, France.

* [Weiss M. and Baret F.,2020](https://step.esa.int/docs/extra/ATBD_S2ToolBox_L2B_V2.0.pdf). S2ToolBox Level 2 Products: LAI, FAPAR, FCOVER, 2.0. ed. Institut National de la Recherche Agronomique, Avignon, France.

#### Dependencies:
- rasterio 1.3.9
- matplotlib 3.7.2
- datetime 5.4
- skimage 0.20.0
- scipy 1.11.1
- pickle 0.0.12
- tqdm 4.65.0

### Initiation

In [1]:
from tools import SL2P
from tools import read_sentinel2_safe_image
from tools import dictionariesSL2P 
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import os,numpy
import rasterio
import zipfile
import shutil

In [2]:
from tools import SL2PBF 
from tools import SL2PNF 
from tools import SL2POTHER 

In [3]:
# Specify output variable, input satellite imagery and location of downscaled images in SAFE format
# The downscaled images are produced using
variableName='LAI'
imageCollectionName="S2_SR"
path = 'F:/novascotia/DownscaledV0/'


In [4]:
#### Input data Control ***-
if variableName not in dictionariesSL2P.make_net_options().keys():
    raise ValueError(('Supported vegetation valiables: %s'%(dictionariesSL2P.make_net_options().keys())))
if imageCollectionName not in dictionariesSL2P.make_net_options()[variableName].keys():
    raise ValueError(('Supported EO datasets: %s'%(dictionariesSL2P.make_net_options()[variableName].keys())))  
    
### Determine data resolution (we force to 10m)
RES=dictionariesSL2P.define_input_resolution()[imageCollectionName]
RES=10

In [ ]:


# Process all input images using SL2P-CCRS with all three networks 
# SL2PNF - needleaf forests, SL2PBF - broadleaf forests, SL2POTHER - all the rest
# You need to combine these after using 10m land cover maps
# Make sure to mask non clear sky pixels yourself based on the input image
inputFileList = os.listdir(path)
inputFileList = [k for k in inputFileList if '.SAFE' in k]
for fileName in inputFileList :
    print(fileName)

    #read image in
    s2=read_sentinel2_safe_image.read_s2(path+fileName,res=RES)        

    #prepare image for sl2p
    sl2p_inp=SL2P.prepare_sl2p_inp(s2,variableName,imageCollectionName)

    #run algorithm
    algorithm = SL2PBF
    varmap=SL2P.SL2P(algorithm,sl2p_inp,variableName,imageCollectionName)

    #export product
    profile=s2['profile']
    profile.update({'count':5,'dtype':'uint8','driver':'GTiff'}) 
    outputFileName = path+fileName.replace('.SAFE','_%s_%s.tif'%(variableName,algorithm.__name__.split('.')[-1]))
    with rasterio.open(outputFileName,'w',**profile) as dst:
        dst.write(varmap[variableName]*30,1)
        dst.write(varmap[variableName+'_uncertainty']*30,2)
        dst.write(varmap['sl2p_inputFlag'],3)
        dst.write(varmap['sl2p_outputFlag'],4)
        dst.write(s2['SCL'],5)
        
    #run algorithm
    algorithm = SL2PNF
    varmap=SL2P.SL2P(algorithm,sl2p_inp,variableName,imageCollectionName)

    #export product
    profile=s2['profile']
    profile.update({'count':5,'dtype':'uint8','driver':'GTiff'}) 
    outputFileName = path+fileName.replace('.SAFE','_%s_%s.tif'%(variableName,algorithm.__name__.split('.')[-1]))
    with rasterio.open(outputFileName,'w',**profile) as dst:
        dst.write(varmap[variableName]*30,1)
        dst.write(varmap[variableName+'_uncertainty']*30,2)
        dst.write(varmap['sl2p_inputFlag'],3)
        dst.write(varmap['sl2p_outputFlag'],4)
        dst.write(s2['SCL'],5)
        
    #run algorithm
    algorithm = SL2POTHER
    varmap=SL2P.SL2P(algorithm,sl2p_inp,variableName,imageCollectionName)

    #export product
    profile=s2['profile']
    profile.update({'count':5,'dtype':'uint8','driver':'GTiff'}) 
    outputFileName = path+fileName.replace('.SAFE','_%s_%s.tif'%(variableName,algorithm.__name__.split('.')[-1]))
    with rasterio.open(outputFileName,'w',**profile) as dst:
        dst.write(varmap[variableName]*30,1)
        dst.write(varmap[variableName+'_uncertainty']*30,2)
        dst.write(varmap['sl2p_inputFlag'],3)
        dst.write(varmap['sl2p_outputFlag'],4)
        dst.write(s2['SCL'],5)